# EDA BIOMETRICO

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [3]:
ruta ="dataset/produccion08.xls"
df_cota = pd.read_excel(ruta)

WARNING *** file size (1988229) not 512 + multiple of sector size (512)


### BD info produccion

In [4]:
ruta2 ="empleados/info producciòn.xlsx"
df_empleados = pd.read_excel(ruta2)

In [5]:
df_cota.isnull().sum()

Tiempo                  0
ID de Usuario         990
Nombre                990
Apellido             1185
Número de tarjeta    5615
Dispositivo             0
Punto del evento        0
Verificación            0
Estado                  0
Evento                  0
Notas                5802
dtype: int64

### LIMPIEZA

In [6]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df_cota.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df_cota.columns = new_name


df_cota.duplicated().sum()


np.int64(0)

In [7]:
#cambia el nombre de la columna de numero a ID del trabajador
df_cota.rename(columns={'número': 'id'}, inplace=True) 

df_cota["nombre_completo"] = df_cota["nombre"] +" "+df_cota["apellido"]

### MATCH TRABAJADORES BIOMETRICO

In [8]:
from fuzzywuzzy import process, fuzz
import pandas as pd
import unidecode

# ---- Función de normalización ----
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    s = str(nombre).upper().strip()
    s = unidecode.unidecode(s)
    s = " ".join(s.split())
    return s

# ---- Normalizar columnas ----
df_cota['nombre_limpio']     = df_cota['nombre_completo'].apply(limpiar_nombre)
df_empleados['empleado_limpio'] = df_empleados['Empleado'].apply(limpiar_nombre)

# ---- Listas únicas útiles (sin cadenas vacías) ----
cotas_unique = [n for n in df_cota['nombre_limpio'].unique() if n and len(n) >= 3]
# Usamos el df_empleados tal cual para mantener Departamento y nombre original
THRESHOLD = 85  # ajustar si quieres más/menos estricto

# ---- Buscamos, por cada empleado oficial, su mejor match en df_cota ----
# Guardamos el mejor match por nombre_cota (para evitar duplicados incorrectos)
best_map = {}   # key = nombre_limpio (df_cota), value = dict(oficial, dept, score, empleado_limpio_oficial)

for _, row in df_empleados.iterrows():
    oficial_limpio = row['empleado_limpio']
    if not oficial_limpio:
        continue
    dept = row['Departamento']
    # buscar mejor match en cotas_unique usando token_set_ratio
    resultado = process.extractOne(oficial_limpio, cotas_unique, scorer=fuzz.token_set_ratio)
    if resultado:
        match_name, score = resultado[0], resultado[1]  # extractOne con lista devuelve (match, score)
        # solo consideramos si pasa el umbral
        if score >= THRESHOLD:
            prev = best_map.get(match_name)
            # si ya hay un candidato para ese nombre, quedarnos con el de mayor score
            if (prev is None) or (score > prev['score']):
                best_map[match_name] = {
                    'oficial': row['Empleado'],         # nombre oficial legible
                    'empleado_limpio_oficial': oficial_limpio,
                    'dept': dept,
                    'score': score
                }

# ---- Construir mapas y aplicarlos al df_cota ----
map_depto   = {k: v['dept']     for k, v in best_map.items()}
map_oficial = {k: v['oficial']  for k, v in best_map.items()}

df_cota['Departamento']    = df_cota['nombre_limpio'].map(map_depto)
df_cota['empleado_matched'] = df_cota['nombre_limpio'].map(map_oficial)

# ---- Informes rápidos para diagnosticar ----
print("Oficiales (total):", df_empleados.shape[0])
print("Nombres únicos en df_cota:", len(cotas_unique))
print("Nombres de df_cota asignados a oficiales:", len(map_depto))
print("Umbral usado (THRESHOLD):", THRESHOLD)

empleados_matcheados = sorted({v['empleado_limpio_oficial'] for v in best_map.values()})
empleados_sin_match = df_empleados[
    ~df_empleados['empleado_limpio'].isin(empleados_matcheados)
]['Empleado'].tolist()

print("Empleados oficiales sin match:", len(empleados_sin_match))
print(empleados_sin_match)




/home/donsson/Biometricos/.entorno/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Oficiales (total): 37
Nombres únicos en df_cota: 60
Nombres de df_cota asignados a oficiales: 37
Umbral usado (THRESHOLD): 85
Empleados oficiales sin match: 0
[]


In [9]:
print(f"El numero de trabajadores unicos es de ",df_cota['nombre_limpio'].nunique())

El numero de trabajadores unicos es de  61


In [10]:
df_cota_produccion = df_cota[df_cota["Departamento"].isin(["DIRECCIÓN DE PLANTA","PRODUCCION","MANTENIMIENTO"])]

In [11]:
print(f"Numero de trabajadores unicos en planta",df_cota_produccion['nombre_limpio'].nunique())

Numero de trabajadores unicos en planta 37


### HORAS IDEALES DEL MES

In [12]:
import pandas as pd
import holidays

def horas_ideales_mes(year=2025, month=8):
    # Festivos de Colombia
    co_holidays = holidays.CountryHoliday("CO", years=[year])

    # Rango de días del mes
    inicio = pd.Timestamp(year, month, 1)
    fin = inicio + pd.offsets.MonthEnd(0)
    dias = pd.date_range(inicio, fin, freq="D")

    total_horas = 0.0
    for d in dias:
        if d.weekday() <= 3 and d.date() not in co_holidays:   # Lunes-Jueves
            total_horas += 9.5
        elif d.weekday() == 4 and d.date() not in co_holidays: # Viernes
            total_horas += 8.5
        # sábados, domingos y festivos = 0

    return round(total_horas, 2)

# Ejemplo: horas ideales agosto 2025
print(F"Esta la cantidad de horas reales esperadas para el mes",horas_ideales_mes(2025, 8))
horas_del_mes = horas_ideales_mes(2025, 8)


Esta la cantidad de horas reales esperadas para el mes 175.5


In [ ]:
import pandas as pd
import numpy as np
from datetime import time, date
import holidays

def _is_holiday(d: date, holidays_obj) -> bool:
    if holidays_obj is None:
        return False
    try:
        return d in holidays_obj
    except Exception:
        return d in set(holidays_obj)

def horas_ideales_mes(year=2025, month=8, holidays_obj=None, treat_holidays_as_non_working=True):
    inicio = pd.Timestamp(year, month, 1)
    fin = inicio + pd.offsets.MonthEnd(0)
    dias = pd.date_range(inicio, fin, freq="D").date

    total = 0.0
    for d in dias:
        if d.weekday() <= 3:     # Lun-Jue
            jornada = 9.5
        elif d.weekday() == 4:   # Viernes
            jornada = 8.5
        else:
            jornada = 0.0

        if treat_holidays_as_non_working and _is_holiday(d, holidays_obj) and jornada > 0:
            continue
        total += jornada

    return round(total, 2)

def horas_trabajadas_diario_y_mensual(df, year=2025, month=8,
                                      restar_almuerzo=False, minutos_almuerzo=30,
                                      holidays_obj=None, treat_holidays_as_non_working=True,
                                      debug=False):
    dfx = df.copy()

    if not pd.api.types.is_datetime64_any_dtype(dfx['tiempo']):
        dfx['tiempo'] = pd.to_datetime(dfx['tiempo'], errors='coerce')

    dfx = dfx[dfx['tiempo'].notna()].copy()
    dfx = dfx[(dfx['tiempo'].dt.year == year) & (dfx['tiempo'].dt.month == month)]

    dfx['fecha'] = dfx['tiempo'].dt.date

    agg = (
        dfx.groupby(['nombre_limpio', 'fecha'], as_index=False)
        .agg(
            primera=('tiempo', 'min'),
            ultima=('tiempo', 'max'),
            marcaciones=('tiempo', 'count')
        )
    )

    dept_first = (
        dfx.sort_values(['nombre_limpio', 'fecha', 'tiempo'])
        .groupby(['nombre_limpio', 'fecha'], as_index=False)['Departamento']
        .first()
    )
    agg = agg.merge(dept_first, on=['nombre_limpio', 'fecha'], how='left')

    span_horas = (agg['ultima'] - agg['primera']).dt.total_seconds() / 3600.0
    span_horas = span_horas.where(agg['marcaciones'] >= 2, 0.0)

    if restar_almuerzo:
        span_horas = np.maximum(0.0, span_horas - (minutos_almuerzo / 60.0))

    agg['horas_trabajadas'] = span_horas

    hora_inicio = time(6, 40)
    agg['llegada'] = np.where(
        agg['primera'].isna(),
        "Sin marcación",
        np.where(agg['primera'].dt.time > hora_inicio, "Tarde", "A tiempo")
    )

    # Diario
    diario = agg.rename(columns={'nombre_limpio': 'empleado', 'fecha': 'fecha_dia'})[
        ['empleado', 'Departamento', 'fecha_dia', 'marcaciones', 'primera', 'ultima',
         'horas_trabajadas', 'llegada']
    ].copy()

    # Resumen mensual
    mensual = (
        diario.groupby(['empleado', 'Departamento'], as_index=False)
        .agg(
            horas_trabajadas_total=('horas_trabajadas', 'sum'),
            dias_con_registros=('fecha_dia', 'nunique')
        )
    )
    mensual['horas_trabajadas_total'] = mensual['horas_trabajadas_total'].round(2)

    horas_ideales_const = horas_ideales_mes(year, month, holidays_obj=holidays_obj,
                                           treat_holidays_as_non_working=treat_holidays_as_non_working)
    mensual['horas_ideales'] = horas_ideales_const

    #  Nuevo campo con el total de horas del mes
    mensual['horas_del_mes'] = horas_ideales_const

    mensual['cumplimiento'] = np.where(
        mensual['horas_trabajadas_total'] >= mensual['horas_ideales'], "Sí", "No"
    )

    mensual = mensual[['empleado', 'Departamento', 'horas_trabajadas_total',
                       'horas_ideales', 'cumplimiento', 'dias_con_registros']].sort_values('empleado')

    return diario, mensual, horas_ideales_const




In [ ]:
diario, mensual, horas_ideales_mes = horas_trabajadas_diario_y_mensual(
    df_cota_produccion,
    year=2025,
    month=8,
    holidays_obj=co_holidays,           #festivos 🇨🇴
    treat_holidays_as_non_working=True, # True = los festivos no suman horas
    restar_almuerzo=False              # ponlo True si se quiere el tiempo real de trabajo que por ahora no
)


In [15]:
import xlsxwriter

year = 2025
month = 8

nombre_archivo = f"entregas/mes_produccion_{year}_{month:02d}.xlsx"
with pd.ExcelWriter(nombre_archivo, engine="xlsxwriter") as writer:
    # Guardar las dos hojas
    diario.to_excel(writer, sheet_name="Diario", index=False)
    mensual.to_excel(writer, sheet_name="Mensual", index=False)

    # === AQUI EMPIEZA EL FORMATO CONDICIONAL ===
    workbook = writer.book
    ws_mensual = writer.sheets["Mensual"]


    # Columnas clave
    col_cumplimiento = mensual.columns.get_loc("cumplimiento") + 1  # +1 por índice Excel
    col_dias = mensual.columns.get_loc("dias_con_registros") + 1

    ultima_fila = len(mensual) + 1  # +1 por el encabezado

# HAY UNA REGLA QUE NECESITO QUE BRAYAN ME PASE POR ESCRITO EN EL TEMA DE LAS MARCACIONES PARA ENTENDER COMO HACER EL ANALISIS DE UNA MANERA MAS JUSTA 